In [1]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/29 16:30:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# functions to stop/start a stopwatch

runs = {}

def stopwatch(run: str):
    # new run
    if run not in runs:
        runs[run] = {'start_time': time.time(), 'total_time': None}

    else:
        # incomplete run
        if runs[run]['total_time'] is None:
            run_time = time.time() - runs[run]['start_time']
            print(f"--- {run_time} seconds ---")
            runs[run]['total_time'] = run_time
        # overwrite old run
        else:
            runs[run] = {'start_time': time.time(), 'total_time': None}

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# save the data locally (for later comparisons)

df.write.mode('overwrite').parquet('data/unpartitioned/')

In [6]:
# read in the local df

local_df = spark.read.parquet("data/unpartitioned/")

local_df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.
local_df.createOrReplaceTempView("home_sales")

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

stopwatch('four_bedrooms')

avg_price_query = """
SELECT YEAR(date) as year, 
       ROUND(AVG(price), 2) as avg_price,
       count(*) as n
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

result = spark.sql(avg_price_query)

result.show()

stopwatch('four_bedrooms')

+----+---------+----+
|year|avg_price|   n|
+----+---------+----+
|2019| 300263.7|3120|
|2020|298353.78|3155|
|2021|301819.44|3169|
|2022|296363.88|1394|
+----+---------+----+

--- 0.5538229942321777 seconds ---


In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

stopwatch('three_bedrooms_and_baths')

avg_price_query = """
SELECT YEAR(date) as year, 
       ROUND(AVG(price), 2) as avg_price,
       count(*) as n
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

result = spark.sql(avg_price_query)

result.show()

stopwatch('three_bedrooms_and_baths')

+----+---------+----+
|year|avg_price|   n|
+----+---------+----+
|2019|287287.82|1568|
|2020|294204.16|1553|
|2021|294211.46|1571|
|2022|292725.69| 704|
+----+---------+----+

--- 0.1469287872314453 seconds ---


In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

stopwatch('three_bedrooms_and_baths_and_two_floors')

avg_price_query2 = """
SELECT YEAR(date) as year, 
       ROUND(AVG(price), 2) as avg_price,
       count(*) as n
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
AND sqft_lot >= 2000
AND floors = 2
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

result = spark.sql(avg_price_query2)

result.show()

stopwatch('three_bedrooms_and_baths_and_two_floors')

+----+---------+---+
|year|avg_price|  n|
+----+---------+---+
|2019|282752.72|801|
|2020|293176.41|765|
|2021|290319.06|794|
|2022|287162.18|359|
+----+---------+---+

--- 0.15682506561279297 seconds ---


In [11]:
# 6. What is the "view" rating for homes, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

stopwatch('view_rating_over350k')

view_query = """
SELECT YEAR(date) as year, 
       ROUND(AVG(view), 2) as avg_view,
       count(*) as n
FROM home_sales
WHERE price >=350000
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

result = spark.sql(view_query)

result.show()

stopwatch('view_rating_over350k')

+----+--------+----+
|year|avg_view|   n|
+----+--------+----+
|2019|   32.77|3201|
|2020|   31.94|3220|
|2021|   32.39|3246|
|2022|   31.59|1458|
+----+--------+----+

--- 0.1360311508178711 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

stopwatch('cached_view_rating_over350k')

result = spark.sql(view_query)
result.show()

stopwatch('cached_view_rating_over350k')


+----+--------+----+
|year|avg_view|   n|
+----+--------+----+
|2019|   32.77|3201|
|2020|   31.94|3220|
|2021|   32.39|3246|
|2022|   31.59|1458|
+----+--------+----+

--- 0.18260407447814941 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.partitionBy("date_built").mode('overwrite').parquet('data/partitioned/')

In [16]:
# 11. Read the formatted parquet data.
partitioned_df = spark.read.parquet("data/partitioned/")

partitioned_df.show(10)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [17]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_partitioned")

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

stopwatch('partitioned_view_rating_over350k')

p_view_query = """
SELECT YEAR(date) as year, 
       ROUND(AVG(view), 2) as avg_view,
       count(*) as n
FROM home_sales_partitioned
WHERE price >=350000
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

result = spark.sql(p_view_query)
result.show()

stopwatch('partitioned_view_rating_over350k')

+----+--------+----+
|year|avg_view|   n|
+----+--------+----+
|2019|   32.77|3201|
|2020|   31.94|3220|
|2021|   32.39|3246|
|2022|   31.59|1458|
+----+--------+----+

--- 0.12870502471923828 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
local_df.unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [20]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

In [21]:
# summarise all run times
[print(f"{runs[x]['total_time']:.3f}   {x}") for x in runs]

0.554   four_bedrooms
0.147   three_bedrooms_and_baths
0.157   three_bedrooms_and_baths_and_two_floors
0.136   view_rating_over350k
0.183   cached_view_rating_over350k
0.129   partitioned_view_rating_over350k


[None, None, None, None, None, None]